In [28]:
# set root dir in system path
import sys
sys.path.insert(0, '/Users/dish/Code/codeQual')
print(sys.path)
from codeQual import set_device
device = set_device.set()

['/Users/dish/Code/codeQual', '/Users/dish/Code/codeQual', '/Users/dish/Code/codeQual', '/Users/dish/Code/codeQual', '/Users/dish/Code/codeQual', '../..', '/Users/dish/Code/codeQual/notebooks', '/Users/dish/anaconda3/envs/codequal/lib/python310.zip', '/Users/dish/anaconda3/envs/codequal/lib/python3.10', '/Users/dish/anaconda3/envs/codequal/lib/python3.10/lib-dynload', '', '/Users/dish/anaconda3/envs/codequal/lib/python3.10/site-packages', '/var/folders/52/649zlcmd67lcyfjp21c25b5m0000gn/T/tmpp15hgu5m']


In [29]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = '/Users/dish/Code/codeQual/training_output/checkpoint-232'
num_classes = 3
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, max_length=512).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

0

In [30]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict, load_dataset
code_qual_dataset = DatasetDict.load_from_disk('../data/CodeQualData/code_qual_dataset')

def tokenize(batch):
    return tokenizer(batch["submission"], padding=True, truncation=True)


codequal_encoded: DatasetDict = code_qual_dataset.map(tokenize, batched=True, batch_size=None)
codequal_encoded

DatasetDict({
    train: Dataset({
        features: ['submission', 'label', 'input_ids', 'attention_mask'],
        num_rows: 461
    })
    validation: Dataset({
        features: ['submission', 'label', 'input_ids', 'attention_mask'],
        num_rows: 116
    })
    test: Dataset({
        features: ['submission', 'label', 'input_ids', 'attention_mask'],
        num_rows: 145
    })
})

In [31]:
from torch.nn.functional import cross_entropy
import torch

def forward_pass_with_label(batch):
    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device),
                             reduction="none")
    # Place outputs on CPU for compatibility with other dataset columns
    return {"loss": loss.cpu().numpy(),
            "predicted_label": pred_label.cpu().numpy()}

In [32]:
# Convert our dataset back to PyTorch tensors

codequal_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
# Compute loss values
codequal_encoded["validation"] = codequal_encoded["validation"].map(forward_pass_with_label, batched=True, batch_size=16)
codequal_encoded

Map: 100%|██████████| 116/116 [00:02<00:00, 45.09 examples/s]


DatasetDict({
    train: Dataset({
        features: ['submission', 'label', 'input_ids', 'attention_mask'],
        num_rows: 461
    })
    validation: Dataset({
        features: ['submission', 'label', 'input_ids', 'attention_mask', 'loss', 'predicted_label'],
        num_rows: 116
    })
    test: Dataset({
        features: ['submission', 'label', 'input_ids', 'attention_mask'],
        num_rows: 145
    })
})

In [34]:
codequal_encoded.set_format("pandas")
cols = ["submission", "label", "predicted_label", "loss"]
df_test = codequal_encoded["validation"][:][cols]
df_test

,submission,label,predicted_label,loss
0,# -*- coding: utf-8 -*-\n\ndef answer(s):\n ...,2,1,1.146564
1,"x, y = map(int, input().split())\ndif = abs(ab...",1,1,0.212066
2,"n = int(input())\na = list(map(int, input().sp...",0,0,0.237920
3,word = input()\nif word.endswith('s'):\n prin...,1,1,0.584925
4,"n=int(input())\ns=list(map(int,input().split()...",1,0,1.759817
...,...,...,...,...
111,N = int(input())\nS = [int(input()) for _ in r...,1,1,0.211377
112,"n=int(input())\na=list(map(int,input().split()...",1,1,0.699269
113,word = input()\nif (word.endswith('s')):\n ...,1,1,0.579420
114,line = input()\nline2 = input()\ncount = 0\nif...,1,1,0.401516


In [35]:
df_test.sort_values("loss", ascending=False).head(10)

,submission,label,predicted_label,loss
83,def main():\n import sys\n input = sys.s...,2,1,3.627894
93,"a, b = map(int, input().split())\n\na -= 1\nb ...",2,1,2.916127
15,import sys\n\nread = sys.stdin.read\nreadline ...,2,1,2.797327
115,import sys\nimport itertools\nsys.setrecursion...,2,1,2.163872
95,import sys\ninput = sys.stdin.readline\nsys.se...,2,1,2.055897
47,"N =int(input())\nList = list(map(int, input()....",1,0,1.998440
97,import sys\n\n\n# \n\ndef input():\n return...,0,1,1.963489
61,from sys import stdin\nn = stdin.readline().rs...,1,0,1.963151
46,"n = int(input())\na = list(map(int, input().sp...",1,0,1.857147
98,"n = int(input())\na = list(map(int, input().sp...",1,0,1.842110


In [36]:
df_test.sort_values("loss", ascending=True).head(10)

,submission,label,predicted_label,loss
19,"# 28\nN = int(input())\nAn = list(map(int, inp...",0,0,0.140794
76,import sys\nn = int(input())\na = list(map(int...,0,0,0.144307
42,digit = int(input())\nli = list(input().split(...,0,0,0.151260
94,"N = int(input())\nA = list(map(int, input().sp...",0,0,0.152772
68,n = int(input())\nnum = [int(x) for x in input...,0,0,0.153069
99,"n = int(input())\na = list(map(int, input().sp...",0,0,0.154628
53,"\nn = int(input())\nx = list(map(int,input().s...",0,0,0.156221
24,"import sys\n\nread=sys.stdin.read\nn, *a=map(i...",0,0,0.158462
39,"N = int(input())\nls = list(map(int,input().sp...",0,0,0.158665
9,"n=int(input())\nn_list=list(map(int,input().sp...",0,0,0.164693
